In [48]:
import pandas as pd
import os

In [49]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")
inclusion_root = os.path.join(filepath_root, "Inclusion_Codes") #inclusion code location
filtered_root = os.path.join(filepath_root, "Filtered_Files") #Filtered data location (Filtered data aggregated all diagnosis by patient)
unfiltered_root = os.path.join(filepath_root, "All_csv_files") #unfiltered data location
combined_root = os.path.join(filepath_root, "LBP_X_Files") #LBP + X filtered location


os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(inclusion_root,'inclusion_codes_LBP.xlsx')
path_inclusion_crit_DJD = os.path.join(inclusion_root,'inclusion_codes_DJD.xlsx')
path_inclusion_crit_fracture = os.path.join(inclusion_root,'inclusion_codes_fracture.xlsx')
path_inclusion_crit_other = os.path.join(inclusion_root,'inclusion_codes_other.xlsx')
path_inclusion_crit_scoliosis = os.path.join(inclusion_root,'inclusion_codes_scoliosis.xlsx')

In [50]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [51]:
LBP_filename = 'LBP_all'
other_filename = 'other_all'
scoliosis_filename = 'scoliosis_all'
LBP_all_data = pd.read_csv(unfiltered_root + '/' + LBP_filename + '.csv', encoding = 'utf8',dtype=str)
other_all_data = pd.read_csv(unfiltered_root + '/' + other_filename + '.csv', encoding = 'utf8',dtype=str)
scoliosis_all_data = pd.read_csv(unfiltered_root + '/' + scoliosis_filename + '.csv', encoding = 'utf8',dtype=str)

In [52]:
print(LBP_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [53]:
#Select columns that are needed and add empty columns with diagnosis combined & LBP checker
LBP_diagcode = LBP_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
LBP_diagcode['Diagnosis_combined'] = str(0)
LBP_diagcode['LBP?'] = str(0)
print(LBP_diagcode.keys())

LBP_diagcode_reference = LBP_diagcode



Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/3554974049.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/3554974049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [54]:
#Combine LBP icd codes for same patient
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [55]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [56]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

#other
incl_cpts_other = get_cpts_fromfile(path_inclusion_crit_other)
incl_cpts_text_other = '|'.join(incl_cpts_other.str.strip())

#scoliosis
incl_cpts_scoliosis = get_cpts_fromfile(path_inclusion_crit_scoliosis)
incl_cpts_text_scoliosis = '|'.join(incl_cpts_scoliosis.str.strip())

#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

#other
incl_icds_other = get_icds_fromfile(path_inclusion_crit_other)
incl_icds_text_other = '|'.join(incl_icds_other.str.strip())

#scoliosis
incl_icds_scoliosis = get_icds_fromfile(path_inclusion_crit_scoliosis)
incl_icds_text_scoliosis = '|'.join(incl_icds_scoliosis.str.strip())

In [57]:
#LBP checker 
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/1462156090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [58]:
other_diagcode = other_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
other_diagcode['Diagnosis_combined'] = str(0)
other_diagcode['other?'] = str(0)
print(other_diagcode.keys())

other_diagcode_reference = other_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'other?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/1774246549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/1774246549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_diagcode['other?'] = str(0)


In [59]:
other_diagcode.loc[0,'Diagnosis_combined'] = other_diagcode.loc[0,'Code.dia']
for i in range(1,len(other_diagcode)):
    if (other_diagcode.loc[i,'EMPI'] == other_diagcode.loc[i-1,'EMPI']):
        if other_diagcode.loc[i,'Code.dia'] != other_diagcode.loc[i-1,'Code.dia']:
            other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i,'Code.dia'] + ', ' + str(other_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i,'Code.dia']
        continue;

In [60]:
other_filtered = other_diagcode[other_diagcode['Helper'].notna()]
other_filtered_save = other_filtered

In [61]:
temp_df_other = pd.DataFrame
temp_df_other = other_filtered['Diagnosis_combined'].str.contains(incl_icds_text_other, regex=True)

other_filtered.index = range(len(other_filtered.index))

for i in range(len(temp_df_other)):
    if temp_df_other.iloc[i] == True:
        other_filtered.loc[i,'other?'] = 'other'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/915588701.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_filtered.loc[i,'other?'] = 'other'


Scoliosis

In [62]:
scoliosis_diagcode = scoliosis_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
scoliosis_diagcode['Diagnosis_combined'] = str(0)
scoliosis_diagcode['scoliosis?'] = str(0)
print(scoliosis_diagcode.keys())

scoliosis_diagcode_reference = scoliosis_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'scoliosis?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/3493048040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/3493048040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_diagcode['scoliosis?'] = str(0)


In [63]:
scoliosis_diagcode.loc[0,'Diagnosis_combined'] = scoliosis_diagcode.loc[0,'Code.dia']
for i in range(1,len(scoliosis_diagcode)):
    if (scoliosis_diagcode.loc[i,'EMPI'] == scoliosis_diagcode.loc[i-1,'EMPI']):
        if scoliosis_diagcode.loc[i,'Code.dia'] != scoliosis_diagcode.loc[i-1,'Code.dia']:
            scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i,'Code.dia'] + ', ' + str(scoliosis_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i,'Code.dia']
        continue;

In [64]:
scoliosis_filtered = scoliosis_diagcode[scoliosis_diagcode['Helper'].notna()]
scoliosis_filtered_save = scoliosis_filtered

In [65]:
scoliosis_diagcode.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,scoliosis?
0,100002085,89,White,27.92,72100,M41.9,NaN,M41.9,0
1,100002085,89,White,27.92,72100,M41.9,useful,M41.9,0
2,100004276,92,Other,21.15,72100,M41.9,NaN,M41.9,0
3,100004276,92,Other,21.15,72100,M41.9,useful,M41.9,0
4,100006144,78,Black,22.81,72100,M41.9,NaN,M41.9,0
5,100006144,78,Black,22.81,72100,M41.9,NaN,M41.9,0
6,100006144,78,Black,22.81,72100,M40.12,NaN,"M40.12, M41.9",0
7,100006144,78,Black,22.81,72100,M40.12,NaN,"M40.12, M41.9",0
8,100006144,78,Black,22.81,72100,M40.12,NaN,"M40.12, M41.9",0
9,100006144,78,Black,22.81,72100,M40.12,NaN,"M40.12, M41.9",0


In [66]:
temp_df_scoliosis = pd.DataFrame
temp_df_scoliosis = scoliosis_filtered['Diagnosis_combined'].str.contains(incl_icds_text_scoliosis, regex=True)

scoliosis_filtered.index = range(len(scoliosis_filtered.index))

for i in range(len(temp_df_scoliosis)):
    if temp_df_scoliosis.iloc[i] == True:
        scoliosis_filtered.loc[i,'scoliosis?'] = 'scoliosis'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/1216154863.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_filtered.loc[i,'scoliosis?'] = 'scoliosis'


In [67]:
temp_df_scoliosis.head()

1     True
3     True
10    True
12    True
13    True
Name: Diagnosis_combined, dtype: bool

In [68]:
#combine LBP and other(stack other after LBP: currently these two are not merged)
LBP_and_other = pd.concat([LBP_filtered,other_filtered])
LBP_and_other_scoliosis = pd.concat([LBP_and_other,scoliosis_filtered])

In [69]:
#reset index & helper (not necessary honestly)
LBP_and_other_scoliosis.index = range(len(LBP_and_other_scoliosis.index))
LBP_and_other_scoliosis['Helper'] = str(0)

In [70]:
LBP_and_other_scoliosis.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_other_scoliosis_copy = LBP_and_other_scoliosis
LBP_and_other_scoliosis.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,scoliosis?
0,100000152,81,White,27.02,72100,M81.0,0,M81.0,NaN,other,NaN
1,100000152,81,White,27.02,74000,M41.9,0,M41.9,NaN,NaN,scoliosis
2,100000152,81,White,27.02,72100,G89.29,0,G89.29,LBP,NaN,NaN
3,100000243,92,Declined,24.73,72040,M54.9,0,M54.9,LBP,NaN,NaN
4,100001760,88,White,NaN,72040,G89.29,0,G89.29,LBP,NaN,NaN
5,100002085,89,White,27.92,72100,M41.9,0,M41.9,NaN,NaN,scoliosis
6,100003506,96,Declined,20.06,72100,M54.9,0,M54.9,LBP,NaN,NaN
7,100004276,92,Other,21.15,72100,M54.9,0,M54.9,LBP,NaN,NaN
8,100004276,92,Other,21.15,72100,M41.9,0,M41.9,NaN,NaN,scoliosis
9,100004276,92,Other,17.51,72040,G89.29,0,G89.29,LBP,NaN,NaN


In [71]:
#combining Diagnosis code for LBP and other
#also adding a column to make sure if both LBP and other is included

for i in range(1,len(LBP_and_other_scoliosis)):
    if LBP_and_other_scoliosis.loc[i,'EMPI'] == LBP_and_other_scoliosis.loc[i-1,'EMPI']:
        if LBP_and_other_scoliosis.loc[i,'Diagnosis_combined'] != LBP_and_other_scoliosis.loc[i-1,'Diagnosis_combined']:
            LBP_and_other_scoliosis.loc[i,'Diagnosis_combined'] = LBP_and_other_scoliosis.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_other_scoliosis.loc[i-1,'Diagnosis_combined'])
        if LBP_and_other_scoliosis.loc[i,'LBP?'] != 'LBP':
            if LBP_and_other_scoliosis.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_other_scoliosis.loc[i,'LBP?'] = 'LBP'
        if LBP_and_other_scoliosis.loc[i,'other?'] != 'other':
            if LBP_and_other_scoliosis.loc[i-1,'other?'] == 'other':
                LBP_and_other_scoliosis.loc[i,'other?'] = 'other'
        if LBP_and_other_scoliosis.loc[i,'scoliosis?'] != 'scoliosis':
            if LBP_and_other_scoliosis.loc[i-1,'scoliosis?'] == 'scoliosis':
                LBP_and_other_scoliosis.loc[i,'scoliosis?'] = 'scoliosis'
    else:
        LBP_and_other_scoliosis.loc[i-1,'Helper'] = 'useful'

In [72]:
#filtering out the rows with combined diagnosis(ones that have every diagnosis code)
LBPotherscoliosis = LBP_and_other_scoliosis.loc[LBP_and_other_scoliosis['Helper'] == 'useful']
LBPotherscoliosis.reset_index(drop =True, inplace=True)
LBPotherscoliosis['All'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_16827/2856080226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPotherscoliosis['All'] = str(0)


In [73]:
LBPotherscoliosis.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,scoliosis?,All
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M41.9, M81.0",LBP,other,scoliosis,0
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,NaN,0
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,NaN,0
3,100002085,89,White,27.92,72100,M41.9,useful,M41.9,NaN,NaN,scoliosis,0
4,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,NaN,0


In [74]:
for i in range(0,len(LBPotherscoliosis)):
    if LBPotherscoliosis.loc[i,'LBP?'] == 'LBP':
        if LBPotherscoliosis.loc[i,'other?'] == 'other':
            LBPotherscoliosis.loc[i,'All'] = 'Or'
        if LBPotherscoliosis.loc[i,'scoliosis?'] == 'scoliosis':
            LBPotherscoliosis.loc[i,'All'] = 'Or'

In [75]:
LBPotherscoliosis.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,scoliosis?,All
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M41.9, M81.0",LBP,other,scoliosis,Or
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,NaN,0
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,NaN,0
3,100002085,89,White,27.92,72100,M41.9,useful,M41.9,NaN,NaN,scoliosis,0
4,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,NaN,0
5,100004276,92,Other,17.51,72040,G89.29,useful,"G89.29, M41.9, M54.9",LBP,NaN,scoliosis,Or
6,100004445,71,Black,NaN,74000,M81.0,useful,M81.0,NaN,other,NaN,0
7,100004610,98,White,NaN,72040,M81.0,useful,"M81.0, S30.0XXA",LBP,other,NaN,Or
8,100006144,78,Black,23.33,72100,G89.29,useful,"G89.29, M40.12, M40.12, M41.9, M81.0, M43.27, ...",LBP,other,scoliosis,Or
9,100006348,80,White,27,72040,G89.29,useful,"G89.29, M43.22, M81.0",LBP,other,NaN,Or


In [76]:
#LBP_only = LBP - (LBP & other)
#other_only = other - (LBP & other)
LBP_only = LBPotherscoliosis.loc[LBPotherscoliosis['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['All'] != 'All']
other_only = LBPotherscoliosis.loc[LBPotherscoliosis['other?'] == 'other']
other_only = other_only[other_only['All'] != 'All']
scoliosis_only = LBPotherscoliosis.loc[LBPotherscoliosis['scoliosis?'] == 'scoliosis']
scoliosis_only = scoliosis_only[scoliosis_only['All'] != 'All']
LBPotherscoliosis_only = LBPotherscoliosis.loc[LBPotherscoliosis['All'] == 'Or']

In [77]:
LBPotherscoliosis_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,scoliosis?,All
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M41.9, M81.0",LBP,other,scoliosis,Or
5,100004276,92,Other,17.51,72040,G89.29,useful,"G89.29, M41.9, M54.9",LBP,NaN,scoliosis,Or
7,100004610,98,White,NaN,72040,M81.0,useful,"M81.0, S30.0XXA",LBP,other,NaN,Or
8,100006144,78,Black,23.33,72100,G89.29,useful,"G89.29, M40.12, M40.12, M41.9, M81.0, M43.27, ...",LBP,other,scoliosis,Or
9,100006348,80,White,27,72040,G89.29,useful,"G89.29, M43.22, M81.0",LBP,other,NaN,Or


In [78]:
LBP_and_other_total = pd.concat([LBP_diagcode,other_diagcode])
LBP_and_other_scoliosis_total = pd.concat([LBP_and_other_total,scoliosis_diagcode])
LBP_and_other_scoliosis_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_other_scoliosis_total)

5322

In [79]:
print('Number of Total Patients(LBP+other):  ' + str(len(LBP_and_other_scoliosis_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of other only:  ' + str(len(other_only)))
print('Number of scoliosis only:  ' + str(len(scoliosis_only)))
print('Number of All only:  ' + str(len(LBPotherscoliosis_only)))

#LBP_filtered = LBP
#other_filtered = other
#LBP & other filtered = LBPnother_filtered


Number of Total Patients(LBP+other):  5321
Number of LBP only:  4320
Number of other only:  2251
Number of scoliosis only:  992
Number of All only:  1798


In [80]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)

other_only_csv_path = os.path.join(filtered_root,'other_patients.csv')
other_filtered.to_csv(other_only_csv_path)

In [81]:
other_csv_path = os.path.join(combined_root,'LBP_other_scoliosis_patients.csv')
LBPotherscoliosis_only.to_csv(other_csv_path)